In [20]:
from fetch_data import fetch_events, clear_events, load_csv_and_create_dataframe, load_credentials
import numpy as np
import json
import pandas as pd

from pprint import PrettyPrinter
from matplotlib import pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, Flatten, Activation
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.metrics import mean_squared_error
plt.style.use('dark_background')
from sklearn.metrics import classification_report, confusion_matrix
import random
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16138411449482822683
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10336468590597931805
physical_device_desc: "device: XLA_CPU device"
]


### Parse position data (x, y, w, h) and splits it in separate data series for each dimention

In [21]:
def generate_x_data(data): 
    data['pos'] = data['pos'].apply(lambda x: np.array([list(map(float, value.replace('(','[').replace(')',']').replace('[','').replace(']','').split(', '))) for value in x]))
    data['len'] = data['pos'].apply(len)
    data['x'] = data['pos'].apply(lambda x: [data[0] for data in x])
    data['y'] = data['pos'].apply(lambda x: [data[1] for data in x])
    data['sizex'] = data['pos'].apply(lambda x: [data[2] for data in x])
    data['sizey'] = data['pos'].apply(lambda x: [data[3] for data in x])
    
    return data

### Data fetching

This function fetchs data from multiple days into one dataframe and downloads it's csv's

In [22]:
def fetch_multiple_days (credentials, *days, equalize=False, for_review=False):
    
    purchases, regular_events = clear_events(fetch_events(credentials['user'], credentials['pwd'], days[0]).json()[0], for_review=for_review)
    
    data = load_csv_and_create_dataframe(purchases, 1)
    if equalize:
        data = load_csv_and_create_dataframe(sorted(regular_events, key=lambda x:random.random())[:len(purchases)], 0, data)
    else:
        data = load_csv_and_create_dataframe(regular_events, 0, data)
        
    for day in days[1:]:
        purchases, regular_events = clear_events(fetch_events(credentials['user'], credentials['pwd'], day).json()[0], for_review=for_review)
        data = load_csv_and_create_dataframe(purchases, 1, data)
        if equalize:
            data = load_csv_and_create_dataframe(sorted(regular_events, key=lambda x:random.random())[:len(purchases)], 0, data)
        else:
            data = load_csv_and_create_dataframe(regular_events, 0, data)
    return data

### Data constrain

LSTM's requires setting a specific amount of timesteps, this function set the data to that size, and then formats it to the expected input of an LSTM. ex:

```json
t0 (a0, a1, ... an)
t1 (b0, b1, ... bn)
t2 (c0, c1, ... cn)
```

In [23]:
def constrain_data(data, max_phrase_len):
    X_x_train = pad_sequences(data['x'], maxlen = max_phrase_len) 
    X_y_train = pad_sequences(data['y'], maxlen = max_phrase_len)
    X_szx_train = pad_sequences(data['sizex'], maxlen = max_phrase_len)
    X_szy_train = pad_sequences(data['sizey'], maxlen = max_phrase_len)

    x_train = np.array(list(zip(X_x_train, X_y_train, X_szx_train, X_szy_train)))
    
    return x_train

## Fetching train and test data

In [24]:
load_credentials()
credentials = {}
with open('credentials_visio.json') as creds:
    credentials = json.load(creds)
train_data = fetch_multiple_days(credentials, '2020-02-03','2020-02-02','2020-02-01','2020-01-31', '2020-01-30', '2020-01-28', '2020-01-28', '2020-01-27', '2020-01-25', '2020-01-24', '2020-01-13', '2020-01-20')
test_data = fetch_multiple_days(credentials, '2020-01-30', equalize=False)

Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed
Login succed


In [25]:
train_Data = train_data.copy()
train_Data = generate_x_data(train_Data)
feature_dimention = 4

test_Data = test_data.copy()
test_Data = generate_x_data(test_Data)

max_phrase_len = min(test_Data['len'].max() if test_Data['len'].max() > test_Data['len'].max() else test_Data['len'].max(), 3000)
feature_dimention = 4

In [26]:
test_Data.describe()

,len
count,429.000000
mean,649.121212
std,796.967465
min,92.000000
25%,127.000000
50%,294.000000
75%,904.000000
max,5312.000000


In [27]:
def normalize(arr):
    norm = arr - arr.min()
    norm = norm / np.abs(norm).max()
    
    return norm

In [28]:
batch_size = 50
epochs = 1000

In [29]:
Y_train = train_Data['purchase']
y_train = to_categorical(Y_train)

X_x_train = pad_sequences(train_Data['x'], maxlen = max_phrase_len) 
X_y_train = pad_sequences(train_Data['y'], maxlen = max_phrase_len)
X_szx_train = pad_sequences(train_Data['sizex'], maxlen = max_phrase_len)
X_szy_train = pad_sequences(train_Data['sizey'], maxlen = max_phrase_len)

x_train = np.array(list(zip(X_x_train, X_y_train, X_szx_train, X_szy_train)))

In [30]:
Y_test = test_Data['purchase']
y_test = to_categorical(Y_test)

X_x_test = pad_sequences(test_Data['x'], maxlen = max_phrase_len) 
X_y_test = pad_sequences(test_Data['y'], maxlen = max_phrase_len)
X_szx_test = pad_sequences(test_Data['sizex'], maxlen = max_phrase_len)
X_szy_test = pad_sequences(test_Data['sizey'], maxlen = max_phrase_len)

x_test = np.array(list(zip(X_x_test, X_y_test, X_szx_test, X_szy_test)))

In [31]:
model_lstm = Sequential()
# model_lstm.add(Embedding(input_dim = 4, output_dim = 256, input_length = max_phrase_len))
# model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(64, input_shape=(4, max_phrase_len), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64, input_shape=(4, max_phrase_len), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64, input_shape=(4, max_phrase_len)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(2, activation='softmax'))
#model_lstm.add(Flatten())
model_lstm.summary()

model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['acc']
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4, 64)             784640    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 64)             33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 64)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

In [32]:
history = model_lstm.fit(
    x_train,
    y_train,
    validation_split = 0.1,
    epochs = epochs,
    batch_size = batch_size,
    verbose=1
)

Train on 5326 samples, validate on 592 samples
Epoch 1/1000
5326/5326 [==============================] - 2s 437us/step - loss: 0.3229 - acc: 0.8357 - val_loss: 0.3442 - val_acc: 0.7973
Epoch 2/1000
5326/5326 [==============================] - 1s 238us/step - loss: 0.2907 - acc: 0.8504 - val_loss: 0.3300 - val_acc: 0.8345
Epoch 3/1000
5326/5326 [==============================] - 1s 236us/step - loss: 0.2902 - acc: 0.8534 - val_loss: 0.3191 - val_acc: 0.8345
Epoch 4/1000
5326/5326 [==============================] - 1s 237us/step - loss: 0.2885 - acc: 0.8534 - val_loss: 0.3284 - val_acc: 0.8108
Epoch 5/1000
5326/5326 [==============================] - 1s 236us/step - loss: 0.2871 - acc: 0.8505 - val_loss: 0.3269 - val_acc: 0.8243
Epoch 6/1000
5326/5326 [==============================] - 1s 238us/step - loss: 0.2852 - acc: 0.8543 - val_loss: 0.3258 - val_acc: 0.8142
Epoch 7/1000
5326/5326 [==============================] - 1s 238us/step - loss: 0.2821 - acc: 0.8526 - val_loss: 0.3251 - val

Epoch 118/1000
5326/5326 [==============================] - 1s 253us/step - loss: 0.2616 - acc: 0.8750 - val_loss: 0.3062 - val_acc: 0.8480
Epoch 119/1000
5326/5326 [==============================] - 1s 243us/step - loss: 0.2641 - acc: 0.8650 - val_loss: 0.3130 - val_acc: 0.8446
Epoch 120/1000
5326/5326 [==============================] - 1s 252us/step - loss: 0.2567 - acc: 0.8710 - val_loss: 0.3072 - val_acc: 0.8429
Epoch 121/1000
5326/5326 [==============================] - 1s 248us/step - loss: 0.2564 - acc: 0.8755 - val_loss: 0.3110 - val_acc: 0.8446
Epoch 122/1000
5326/5326 [==============================] - 1s 246us/step - loss: 0.2664 - acc: 0.8650 - val_loss: 0.3149 - val_acc: 0.8412
Epoch 123/1000
5326/5326 [==============================] - 1s 244us/step - loss: 0.2684 - acc: 0.8635 - val_loss: 0.3236 - val_acc: 0.8277
Epoch 124/1000
5326/5326 [==============================] - 1s 241us/step - loss: 0.2663 - acc: 0.8659 - val_loss: 0.3114 - val_acc: 0.8446
Epoch 125/1000
5326/

5326/5326 [==============================] - 1s 247us/step - loss: 0.2561 - acc: 0.8704 - val_loss: 0.3063 - val_acc: 0.8395
Epoch 235/1000
5326/5326 [==============================] - 1s 247us/step - loss: 0.2510 - acc: 0.8757 - val_loss: 0.3022 - val_acc: 0.8480
Epoch 236/1000
5326/5326 [==============================] - 1s 248us/step - loss: 0.2604 - acc: 0.8661 - val_loss: 0.3184 - val_acc: 0.8209
Epoch 237/1000
5326/5326 [==============================] - 1s 249us/step - loss: 0.2573 - acc: 0.8703 - val_loss: 0.3148 - val_acc: 0.8345
Epoch 238/1000
5326/5326 [==============================] - 1s 247us/step - loss: 0.2569 - acc: 0.8708 - val_loss: 0.3215 - val_acc: 0.8226
Epoch 239/1000
5326/5326 [==============================] - 1s 248us/step - loss: 0.2564 - acc: 0.8676 - val_loss: 0.3079 - val_acc: 0.8345
Epoch 240/1000
5326/5326 [==============================] - 1s 249us/step - loss: 0.2546 - acc: 0.8721 - val_loss: 0.3146 - val_acc: 0.8429
Epoch 241/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 241us/step - loss: 0.2477 - acc: 0.8789 - val_loss: 0.3049 - val_acc: 0.8497
Epoch 351/1000
5326/5326 [==============================] - 1s 241us/step - loss: 0.2474 - acc: 0.8719 - val_loss: 0.3063 - val_acc: 0.8463
Epoch 352/1000
5326/5326 [==============================] - 1s 241us/step - loss: 0.2490 - acc: 0.8757 - val_loss: 0.2993 - val_acc: 0.8547
Epoch 353/1000
5326/5326 [==============================] - 1s 243us/step - loss: 0.2495 - acc: 0.8789 - val_loss: 0.3101 - val_acc: 0.8497
Epoch 354/1000
5326/5326 [==============================] - 1s 240us/step - loss: 0.2500 - acc: 0.8744 - val_loss: 0.3166 - val_acc: 0.8463
Epoch 355/1000
5326/5326 [==============================] - 1s 242us/step - loss: 0.2487 - acc: 0.8811 - val_loss: 0.3047 - val_acc: 0.8446
Epoch 356/1000
5326/5326 [==============================] - 1s 243us/step - loss: 0.2489 - acc: 0.8787 - val_loss: 0.3079 - val_acc: 0.8395
Epoch 357/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 244us/step - loss: 0.2483 - acc: 0.8798 - val_loss: 0.3136 - val_acc: 0.8378
Epoch 467/1000
5326/5326 [==============================] - 1s 247us/step - loss: 0.2484 - acc: 0.8793 - val_loss: 0.3054 - val_acc: 0.8395
Epoch 468/1000
5326/5326 [==============================] - 1s 245us/step - loss: 0.2509 - acc: 0.8789 - val_loss: 0.3089 - val_acc: 0.8378
Epoch 469/1000
5326/5326 [==============================] - 1s 243us/step - loss: 0.2533 - acc: 0.8736 - val_loss: 0.3051 - val_acc: 0.8446
Epoch 470/1000
5326/5326 [==============================] - 1s 247us/step - loss: 0.2524 - acc: 0.8744 - val_loss: 0.3010 - val_acc: 0.8378
Epoch 471/1000
5326/5326 [==============================] - 1s 251us/step - loss: 0.2492 - acc: 0.8765 - val_loss: 0.3012 - val_acc: 0.8463
Epoch 472/1000
5326/5326 [==============================] - 1s 251us/step - loss: 0.2468 - acc: 0.8793 - val_loss: 0.2931 - val_acc: 0.8547
Epoch 473/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 252us/step - loss: 0.2499 - acc: 0.8772 - val_loss: 0.3047 - val_acc: 0.8463
Epoch 583/1000
5326/5326 [==============================] - 1s 250us/step - loss: 0.2462 - acc: 0.8789 - val_loss: 0.3086 - val_acc: 0.8514
Epoch 584/1000
5326/5326 [==============================] - 1s 248us/step - loss: 0.2436 - acc: 0.8787 - val_loss: 0.3213 - val_acc: 0.8378
Epoch 585/1000
5326/5326 [==============================] - 1s 254us/step - loss: 0.2466 - acc: 0.8778 - val_loss: 0.3132 - val_acc: 0.8480
Epoch 586/1000
5326/5326 [==============================] - 1s 250us/step - loss: 0.2460 - acc: 0.8736 - val_loss: 0.3150 - val_acc: 0.8378
Epoch 587/1000
5326/5326 [==============================] - 1s 242us/step - loss: 0.2495 - acc: 0.8753 - val_loss: 0.3142 - val_acc: 0.8412
Epoch 588/1000
5326/5326 [==============================] - 1s 244us/step - loss: 0.2484 - acc: 0.8751 - val_loss: 0.3156 - val_acc: 0.8345
Epoch 589/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 254us/step - loss: 0.2380 - acc: 0.8832 - val_loss: 0.3030 - val_acc: 0.8547
Epoch 699/1000
5326/5326 [==============================] - 1s 254us/step - loss: 0.2388 - acc: 0.8868 - val_loss: 0.2939 - val_acc: 0.8598
Epoch 700/1000
5326/5326 [==============================] - 1s 260us/step - loss: 0.2358 - acc: 0.8875 - val_loss: 0.3016 - val_acc: 0.8564
Epoch 701/1000
5326/5326 [==============================] - 1s 244us/step - loss: 0.2364 - acc: 0.8866 - val_loss: 0.2987 - val_acc: 0.8530
Epoch 702/1000
5326/5326 [==============================] - 1s 245us/step - loss: 0.2401 - acc: 0.8872 - val_loss: 0.2990 - val_acc: 0.8446
Epoch 703/1000
5326/5326 [==============================] - 1s 253us/step - loss: 0.2387 - acc: 0.8791 - val_loss: 0.2942 - val_acc: 0.8547
Epoch 704/1000
5326/5326 [==============================] - 1s 254us/step - loss: 0.2399 - acc: 0.8808 - val_loss: 0.2892 - val_acc: 0.8564
Epoch 705/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 255us/step - loss: 0.2352 - acc: 0.8868 - val_loss: 0.2924 - val_acc: 0.8514
Epoch 815/1000
5326/5326 [==============================] - 1s 265us/step - loss: 0.2399 - acc: 0.8830 - val_loss: 0.2951 - val_acc: 0.8514
Epoch 816/1000
5326/5326 [==============================] - 1s 260us/step - loss: 0.2355 - acc: 0.8868 - val_loss: 0.2912 - val_acc: 0.8530
Epoch 817/1000
5326/5326 [==============================] - 1s 274us/step - loss: 0.2372 - acc: 0.8825 - val_loss: 0.2968 - val_acc: 0.8429
Epoch 818/1000
5326/5326 [==============================] - 1s 258us/step - loss: 0.2337 - acc: 0.8883 - val_loss: 0.2893 - val_acc: 0.8581
Epoch 819/1000
5326/5326 [==============================] - 1s 260us/step - loss: 0.2383 - acc: 0.8857 - val_loss: 0.2908 - val_acc: 0.8564
Epoch 820/1000
5326/5326 [==============================] - 1s 277us/step - loss: 0.2423 - acc: 0.8858 - val_loss: 0.3007 - val_acc: 0.8497
Epoch 821/1000
5326/5326 [=========

5326/5326 [==============================] - 1s 257us/step - loss: 0.2402 - acc: 0.8813 - val_loss: 0.3069 - val_acc: 0.8412
Epoch 931/1000
5326/5326 [==============================] - 1s 249us/step - loss: 0.2373 - acc: 0.8840 - val_loss: 0.3125 - val_acc: 0.8412
Epoch 932/1000
5326/5326 [==============================] - 1s 251us/step - loss: 0.2439 - acc: 0.8813 - val_loss: 0.3008 - val_acc: 0.8345
Epoch 933/1000
5326/5326 [==============================] - 1s 255us/step - loss: 0.2409 - acc: 0.8815 - val_loss: 0.3064 - val_acc: 0.8480
Epoch 934/1000
5326/5326 [==============================] - 1s 261us/step - loss: 0.2388 - acc: 0.8825 - val_loss: 0.3034 - val_acc: 0.8463
Epoch 935/1000
5326/5326 [==============================] - 1s 256us/step - loss: 0.2417 - acc: 0.8789 - val_loss: 0.3162 - val_acc: 0.8345
Epoch 936/1000
5326/5326 [==============================] - 1s 263us/step - loss: 0.2429 - acc: 0.8800 - val_loss: 0.3094 - val_acc: 0.8446
Epoch 937/1000
5326/5326 [=========

In [33]:
score = model_lstm.evaluate(x_test, y_test, verbose=1)
model_lstm.save('purchase_classification_20200205.h5')
print (score)

429/429 [==============================] - 0s 96us/step
[0.23039306592552256, 0.8951048851013184]


In [34]:
y_pred = model_lstm.predict(x_test)
purchases = []
for result,_id, compra in zip(y_pred, test_data['id'], test_data['purchase']):
    print (f"{_id} {'Nada' if result[0] > 0.95 else 'Compr' if result[1] > 0.8 else 'Sece'} {'Compr' if compra == 1 else 'Nada'} {result[0]}")

1580407200_022 Sece Compr 0.2664857506752014
1580407200_056 Sece Compr 0.5325844883918762
1580392800_045 Sece Compr 0.3057483732700348
1580395200_029 Sece Compr 0.21140056848526
1580395200_038 Sece Compr 0.2699207663536072
1580395200_040 Compr Compr 0.15642638504505157
1580395200_074 Sece Compr 0.30861037969589233
1580403600_017 Sece Compr 0.7738438844680786
1580404800_000 Sece Compr 0.38707664608955383
1580404800_001 Sece Compr 0.6824271082878113
1580404800_073 Sece Compr 0.47753235697746277
1580406000_072 Sece Compr 0.5504408478736877
1580408400_030 Sece Compr 0.3541993200778961
1580408400_049 Sece Compr 0.20033667981624603
1580408400_053 Sece Compr 0.48832955956459045
1580408400_061 Sece Compr 0.22162388265132904
1580408400_069 Sece Compr 0.9153889417648315
1580412000_104 Sece Compr 0.2736496329307556
1580413200_002 Sece Compr 0.25770294666290283
1580413200_046 Sece Compr 0.5442635416984558
1580413200_071 Nada Compr 0.971476674079895
1580414400_037 Sece Compr 0.25460201501846313
158

In [35]:
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
[['tp', 'fp'],['fn', 'tn']]
print(matrix)

[[314  23]
 [ 22  70]]
